# Amortization Application

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pv = 360000
rate = 0.05875
term = 30
pmt = np.pmt(rate / 12, term * 12, -pv)
extra_principal = 100

In [4]:
def amortize(pv,rate,term,extra_principal=0):
    pmt = np.pmt(rate / 12, term * 12, -pv)
    amort_table = pd.DataFrame({'Balance': np.zeros(term * 12 + 1), 
                            'Interest': np.zeros(term * 12 + 1),
                            'Principal': np.zeros(term * 12 + 1),
                            'Cum_Int' : np.zeros(term * 12 + 1)})
    amort_table['Balance'][0] = pv
    amort_table['Interest'][0] = amort_table['Balance'][0] * rate / 12
    amort_table['Principal'][0] = (pmt + extra_principal) - amort_table['Interest'][0]
    amort_table['Cum_Int'] = amort_table['Interest'].cumsum()
    for payment in range(1,360):
            actual_pmt  = pmt + extra_principal
            amort_table['Balance'][payment] = max(0,
                amort_table['Balance'][payment - 1] - amort_table['Principal'][payment - 1])
            amort_table['Interest'][payment] = rate / 12 * amort_table['Balance'][payment]
            amort_table['Principal'][payment] = actual_pmt - amort_table['Interest'][payment]
            if amort_table['Balance'][payment] < actual_pmt :
                amort_table['Principal'][payment] = amort_table['Balance'][payment]
                break
    amort_table = amort_table[amort_table['Balance'] != 0]
    amort_table['Cum_Int'] = amort_table['Interest'].cumsum()
    amort_table = round(amort_table,2)
    summary = pd.Series({
        'Interest': rate,
        'Payment' : round(pmt,2),
        'Extra Principal' : extra_principal,
        'Total interest' : amort_table['Cum_Int'].max(),
        'Periods' : amort_table.index[-1] + 1
    })    
    
    return amort_table, summary

In [5]:
amort_table, summary = amortize(pv,rate,term)

In [6]:
amort_table.tail()

,Balance,Interest,Principal,Cum_Int
355,10493.06,51.37,2078.16,406529.69
356,8414.90,41.20,2088.34,406570.89
357,6326.56,30.97,2098.56,406601.87
358,4228.00,20.70,2108.84,406622.57
359,2119.16,10.38,2119.16,406632.94


In [7]:
summary

Interest                0.05875
Payment              2129.54000
Extra Principal         0.00000
Total interest     406632.94000
Periods               360.00000
dtype: float64

In [10]:
scenario_1, summary_1 = amortize(pv,rate,term,100)
scenario_2, summary_2 = amortize(pv,rate,term,200)
scenario_3, summary_3 = amortize(pv,.0475,15)


In [11]:
pd.DataFrame([summary_1,summary_2,summary_3])

,Interest,Payment,Extra Principal,Total interest,Periods
0,0.05875,2129.54,100.0,353588.06,321.0
1,0.05875,2129.54,200.0,313980.32,290.0
2,0.04750,2800.19,0.0,144035.08,180.0
